In [25]:
import IPython.display
import pandas
import np_session

In [26]:
session = np_session.Session('1226010097_636740_20221114_platformD1')

Cleanup files before transfer

In [27]:
if session.project.parent.name == 'DR':
    for _ in session.npexp_path.glob('*surface-image*'):
        for replace in ('-right', '-left'):
            if replace in _.name:
                _.replace(_.parent / _.name.replace(replace, ''))
for _ in session.npexp_path.glob('*platform*.bak'):
    _.unlink()
for _ in session.npexp_path.glob('*platform*D0*json'):
    _.unlink()

In [28]:
names = (*session.D1.names, *session.D2.names)
globs = (*session.D1.globs, *session.D2.globs)
paths = (*session.D1.paths, *session.D2.paths)
types = (*session.D1.types, *session.D2.types)

print(session.npexp_path)
display(df := pandas.DataFrame(({'glob': glob, 'exists': path.name if path else None} for glob, path in zip(globs, paths)), index=names))

\\allen\programs\mindscope\workgroups\np-exp\1226010097_636740_20221114


,glob,exists
ephys_raw_data_probe_A,*_probeABC,1226010097_636740_20221114_probeABC
ephys_raw_data_probe_B,*_probeABC,1226010097_636740_20221114_probeABC
ephys_raw_data_probe_C,*_probeABC,1226010097_636740_20221114_probeABC
ephys_raw_data_probe_D,*_probeDEF,1226010097_636740_20221114_probeDEF
ephys_raw_data_probe_E,*_probeDEF,1226010097_636740_20221114_probeDEF
ephys_raw_data_probe_F,*_probeDEF,1226010097_636740_20221114_probeDEF
synchronization_data,*.sync,1226010097_636740_20221114.sync
behavior_tracking,*.behavior.mp4,1226010097_636740_20221114.behavior.mp4
beh_cam_json,*.behavior.json,1226010097_636740_20221114.behavior.json
eye_tracking,*.eye.mp4,1226010097_636740_20221114.eye.mp4


In [29]:
if not all(paths) or not all(df['exists'].values):
    raise Warning('Some files are missing - continue with upload at your own risk!')
else:
    print('All files exist')

All files exist


In [30]:
for _ in paths:
    if _ and str(session.id) not in _.name:
        print(f'Warning! File name does not contain session id: {_.name}')
else:
    print('All filenames contain session id')


All filenames contain session id


Add D1 & D2 files to platform json

In [31]:
session.platform_json.files = (session.D1.files | session.D2.files)

If a file or dir has already been uploaded to lims, the upload job can fail. 

Remove from the upload manifest:

In [32]:
for name, path in zip(names, paths):
    
    if (file_on_lims := (session.lims_path / path.name)).exists():
        
        print(f'File or dir has already been uploaded:\n\t', file_on_lims, '\n')
        
        if name in (files := session.platform_json.files):
            del files[name]
            session.platform_json.files = files
            print(f'\tRemoved from upload manifest\n')
        

File or dir has already been uploaded:
	 \\allen\programs\mindscope\production\dynamicrouting\prod0\specimen_1192695912\ecephys_session_1226010097\1226010097_636740_20221114_probeABC 

	Removed from upload manifest

File or dir has already been uploaded:
	 \\allen\programs\mindscope\production\dynamicrouting\prod0\specimen_1192695912\ecephys_session_1226010097\1226010097_636740_20221114_probeABC 

	Removed from upload manifest

File or dir has already been uploaded:
	 \\allen\programs\mindscope\production\dynamicrouting\prod0\specimen_1192695912\ecephys_session_1226010097\1226010097_636740_20221114_probeABC 

	Removed from upload manifest

File or dir has already been uploaded:
	 \\allen\programs\mindscope\production\dynamicrouting\prod0\specimen_1192695912\ecephys_session_1226010097\1226010097_636740_20221114_probeDEF 

	Removed from upload manifest

File or dir has already been uploaded:
	 \\allen\programs\mindscope\production\dynamicrouting\prod0\specimen_1192695912\ecephys_session_

In [33]:
IPython.display.JSON(session.platform_json.path)

<IPython.core.display.JSON object>

Set up files required for transfer in scratch space

In [34]:
import datetime
import pathlib

scratch = pathlib.Path('//allen/scratch/aibstemp/svc_neuropix/lims_uploads')
scratch.mkdir(parents=True, exist_ok=True)

file_manifest = scratch / f'{session.id}_{datetime.datetime.now():%Y-%m-%d_%H%M}.txt'
file_manifest.touch(exist_ok=True)

log = file_manifest.with_suffix('.log')
log.touch(exist_ok=True)

shell_script = file_manifest.with_suffix('.sh')
shell_script.touch(exist_ok=True)

Write manifest of relative file paths

In [35]:
src_file_list = []
sorted_files = []
for name, path in zip(names, paths):
    
    if path is None or name not in session.platform_json.files:
        continue    
    
    # specify selection of sorted data files to transfer (excl. AP continuous.dat)
    if name in tuple(f'ephys_raw_data_probe_{_}_sorted' for _ in 'ABCDEF'):
        sorted_files = session.D2.paths_sorted_data if not sorted_files else sorted_files
        src_file_list.extend(_ for _ in sorted_files if path in _.parents) #if path is None this should raise 
        continue
    
    src_file_list.append(path)
    
# add platform json last as a sentinel for successful transfer 
src_file_list.append(session.platform_json.path)

if any(missing := tuple(_ for _ in src_file_list if not _.exists())):
    raise FileNotFoundError('Some files do not exist: ', missing)

file_manifest.write_text(
    '\n'.join(_.relative_to(session.npexp_path).as_posix() for _ in src_file_list),
    newline='\n'
    )
print(file_manifest.read_text())

1226010097_636740_20221114.sync
1226010097_636740_20221114.behavior.mp4
1226010097_636740_20221114.behavior.json
1226010097_636740_20221114.eye.mp4
1226010097_636740_20221114.eye.json
1226010097_636740_20221114.face.json
1226010097_636740_20221114.face.mp4
1226010097_636740_20221114.motor-locs.csv
1226010097_636740_20221114_surface-image1.png
1226010097_636740_20221114_surface-image2.png
1226010097_636740_20221114_surface-image3.png
1226010097_636740_20221114_surface-image4.png
1226010097_636740_20221114_surface-image5.png
1226010097_636740_20221114_surface-image6.png
1226010097_636740_20221114_surface-image1.png
1226010097_636740_20221114_surface-image2.png
1226010097_636740_20221114_surface-image3.png
1226010097_636740_20221114_surface-image4.png
1226010097_636740_20221114_surface-image5.png
1226010097_636740_20221114_surface-image6.png
1226010097_636740_20221114.opto.pkl
1226010097_636740_20221114.mapping.pkl
1226010097_636740_20221114.behavior.pkl
1226010097_636740_20221114.areaCla

In [36]:
rsync_cmd = f'rsync -arv --log-file={file_manifest.with_suffix(".log").as_posix()} --files-from={file_manifest.as_posix()} {session.npexp_path.as_posix()} {np_session.INCOMING_ROOT.as_posix()}'
rsync_cmd

'rsync -arv --log-file=//allen/scratch/aibstemp/svc_neuropix/lims_uploads/1226010097_2023-03-18_2253.log --files-from=//allen/scratch/aibstemp/svc_neuropix/lims_uploads/1226010097_2023-03-18_2253.txt //allen/programs/mindscope/workgroups/np-exp/1226010097_636740_20221114 //allen/programs/braintv/production/incoming/neuralcoding'

Write shell script for the transfer

In [37]:
shell_script_cmd = f"""#!/bin/bash
#SBATCH --job-name=npexp_to_incoming                        # Job name
#SBATCH --mail-type=END,FAIL                                # Mail events (NONE, BEGIN, END, FAIL, ALL)
#SBATCH --mail-user=ben.hardcaslt@alleninstitute.org        # Where to send mail  
#SBATCH --ntasks=1                                          # Run on a single CPU
#SBATCH --mem=4gb                                           # Job memory request (per node)
#SBATCH --time=02:00:00                                     # Time limit hrs:min:sec
#SBATCH --output=npexp_to_incoming_%j.log                   # Standard output and error log
#SBATCH --partition braintv                                 # Partition used for processing
#SBATCH --tmp=100M                                          # Request the amount of space your jobs needs on /scratch/fast
 
pwd; hostname; date
 
echo 'Running rsync job on a single thread'

{rsync_cmd}
 
date
"""

shell_script.write_text(shell_script_cmd, newline='\n') 
# if writing on Windows, newline=\r\n by default, which isn't compatible with bash on linux

1241

In [38]:
import fabric
import np_config

login = np_config.fetch('/logins')['svc_neuropix']
with fabric.Connection(
        host='svc_neuropix@hpc-login',
        connect_kwargs={'password': login['password']},
    ) as ssh:
    ssh.run(f'sbatch {shell_script.as_posix()}')

Submitted batch job 8389951


In [39]:
import time

destination_file_list = [np_session.INCOMING_ROOT / _.relative_to(session.npexp_path) for _ in src_file_list]

while not all(_.exists() for _ in destination_file_list):
    print('Waiting for transfer to complete...\r', end='', flush=True)
    time.sleep(10)
    
print('\nTransfer complete')

Waiting for transfer to complete...
Transfer complete


Check platform json for upload

In [40]:
import shutil
dest_platform_json = np_session.INCOMING_ROOT / session.platform_json.path.name
shutil.copy2(session.platform_json.path, dest_platform_json)
IPython.display.JSON(np_session.INCOMING_ROOT / session.platform_json.path.name)

<IPython.core.display.JSON object>

In [41]:
np_session.write_trigger_file(session)
print('Trigger file written')

No platform json found for 1226010097 in incoming dir - lims upload job will fail when triggered: \\allen\programs\braintv\production\incoming\neuralcoding


Trigger file written
